In [1]:
import os
import re
import speech_recognition as sr
from pydub import AudioSegment
from pytube import YouTube
import librosa
import soundfile
from youtube_transcript_api import YouTubeTranscriptApi
from scipy.io.wavfile import read, write

import numpy as np
import time
import math
#from googleapiclient.discovery import build
import whisper_timestamped as whisper
import json
from tqdm.auto import tqdm

C:\ProgramData\anaconda3\envs\find_keyword_inyt\Lib\site-packages\whisper\timing.py:57: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



C:\ProgramData\anaconda3\envs\find_keyword_inyt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Download the Youtube's audios from a `yt_dl_list.txt`


- 1. Get the keyword video link from this website: https://youglish.com/pronounce/reset/english?
- 2. Copy all the Youtube's link which exist the keyword to `yt_dl_list.txt`
- 3. Execute the below block

In [2]:
def onProgress(stream, chunk, remains):
    total = stream.filesize                     # 取得完整尺寸
    percent = (total-remains) / total * 100     # 減去剩餘尺寸 ( 剩餘尺寸會抓取存取的檔案大小 )
    print(f'downloading… {percent:05.2f}', end='\r')  # 顯示進度，\r 表示不換行，在同一行更新

# Download audio from YouTube video and convert to WAV format
def download_youtube_audio_pytube(video_url):
    # Extract video ID from the video URL
    if "youtu.be" in video_url:
        video_id = video_url.split("youtu.be/")[1]
    elif "watch?v=" in video_url:    
        video_id = video_url.split("watch?v=")[1]
    else:
        print("The link is error: {}".format(video_url)) 

    mp3_audio_name = os.path.join('download_raw', (video_id + r'.mp3'))
    wav_audio_name = os.path.join('download_raw', (video_id + r'.wav'))
    #print(mp3_audio_name)
    
    yt = YouTube(video_url, on_progress_callback=onProgress)
    print(yt.title, yt.length, yt.author) # 影片標題, 影片長度 ( 秒 ), 影片作者 
    
    yt.streams.filter().get_audio_only().download(filename=mp3_audio_name)
    
    # Transfer to wav format, because the next process need wav pcm format.
    sound = AudioSegment.from_file(mp3_audio_name)
    sound.export(wav_audio_name, format="wav")
    
    print('Finish!')
    os.remove(mp3_audio_name)
    
    return wav_audio_name

# Start to loop the line in yt list file
dl_count = 0
with open('yt_dl_list.txt') as f:
    for line in f:
       dl_count += 1 
       download_youtube_audio_pytube(line.strip())
       print("Download {} audio!".format(dl_count)) 
print("Finish all download {}.".format(dl_count))


How to see with sound - Jacques S. Abramowicz 316 TED-Ed
Finish!ding… 100.00
Download 1 audio!
How Do Insect Eyes Work? | Compound Eyes | Amazing Animal Senses | SciShow Kids 291 SciShow Kids
Finish!ding… 100.00
Download 2 audio!
Meat-Ghetti & Spag-Balls from American Dad | Botched by Babish 1119 Babish Culinary Universe
Finish!ding… 100.00
Download 3 audio!
The IPCC mitigation report explained in less than 8 minutes feat. @ClimateAdam 462 zentouro
Finish!ding… 100.00
Download 4 audio!
How bumble bees inspired a network of tiny museums | Amanda Schochet 668 TED
Finish!ding… 100.00
Download 5 audio!
Finish all download 5.


## 2. Find & Crop the keyword wav file.
- Update the `search_pattern` to the keyword you want. 
- This section will read the lines in `yt_dl_list.txt` and find the downloaded audio with its transcript

In [24]:
# Set the search pattern for the desired paragraph
search_pattern = r"zoom in"

# Set the path to the ffmpeg binary (required by SpeechRecognition)
#ffmpeg_path = "C:\\Users\\CYCHEN38\\MICRO_ML\\ffmpeg-2023-06-27-git-9b6d191a66-essentials_build\\bin"

In [25]:
# Create a SpeechRecognition Recognizer instance
# recognizer = sr.Recognizer()
dict_log = {} # The log for transcribe_cut_audio & cut_audio_segment

def wave_converter(filename, origin_sr, origin_data, resampled_sr=16000, prefix="resampled_", mix2mono = True):

    '''
    Resample WAV soundfile to a different sample rate.
    
        Input: original sound file
        Output: resampled sound file
        Parameters: 
            - name of the file to be converted, 
            - destination sample rate, default = 16kHz
            - prefix to identify resampled files
            - mix to mono channel, default = False (leave as-is)
            
        Notes: for simplicity it needs to be run in the folder with the files we are converting
    '''    
    
    resampled_file = prefix + filename + '.wav'
    origin_type = origin_data.dtype
    
    # transpose array to librosa shape
    resampled_data = librosa.resample(origin_data.T.astype('float'), orig_sr = origin_sr,  target_sr = resampled_sr) 
    
    if mix2mono == True:
        resampled_data = librosa.to_mono(resampled_data)
    if origin_type != 'int16':
        print("Warning: the data type is changed from {} to int16".format(origin_type))
        resampled_data = resampled_data.T.astype('int16') # transpose back to scipy.io.wavfile shape
    else:
        resampled_data = resampled_data.T.astype(origin_type) # transpose back to scipy.io.wavfile shape
    
    write(resampled_file, resampled_sr, resampled_data)
    #print('Resampled wavefile saved to {}'.format(resampled_file))

# Transcribe audio and grep desired paragraph
def transcribe_cut_audio(audio_file, search_pattern, video_id, dict_single_log, ma_cut):
    #----------------------
    # Basing on the YT transcript to cut the section of search_pattern to a small audio first.
    #----------------------
    t_s = 0
    t_e = 0
    buf_time = 5 # bcs the recognize_google need a long enough audio.
    
    rate, data = read(audio_file)       
    # print(rate, len(data))
    
    # assigning srt with the list of dictionaries obtained by the get_transcript() function
    try:
       srt = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])
    except:
       srt = YouTubeTranscriptApi.get_transcript(video_id, languages=['en-US']) 
    
    for dict_text in srt:
        search_pattern = r'\b{}\b'.format(search_pattern)
        #match = re.findall(pattern, string)
        match = re.search(search_pattern, dict_text['text'], re.IGNORECASE)
        if match:
            t_s = (int)(dict_text['start']) * rate
            t_e = (int)(dict_text['start'] + dict_text['duration']) * rate
            #print('\n')
            #print('Found search_pattern section:')
            #print(dict_text['text'], t_s, t_e)
            #print('\n')
            
    t_s = t_s - rate * buf_time 
    t_e = t_e + rate * buf_time
    dict_single_log['cut_t_s_idx'] = t_s
    dict_single_log['cut_t_e_idx'] = t_e
    # write("{}.wav".format('tempcut'), rate, data[t_s:t_e])
    
    #----------------------
    # Convert to 16khz, mono, int16
    #----------------------
    wave_converter('cut', rate, data[t_s: t_e])

    #----------------------
    # cut_audio_segment
    #----------------------
    keyword_dict = cut_audio_segment(video_id, dict_single_log, ma_cut)
    
    return keyword_dict

def cut_audio_segment(video_id, dict_single_log, ma_cut):    
   audio = whisper.load_audio('resampled_cut.wav')
   model = whisper.load_model("base")
   result = whisper.transcribe(model, audio, language="en")
   #print(json.dumps(result, indent = 2, ensure_ascii = False))
    
   # check if search_pattern is 2 word keyword.
   if ' ' in search_pattern:
       search_pattern_0 = search_pattern.split(' ')[0]
       search_pattern_1 = search_pattern.split(' ')[1]
        
       # Find the match keyword and its start&end time
       t_s = 0
       t_e = 0
       for i, segment in enumerate(result['segments']):
         #print(segment['words'])
         for j, word in enumerate(segment['words']):
             if (search_pattern_0 in word['text'].lower())and(search_pattern_1 in segment['words'][(j+1)]['text'].lower()):
               # Combine the 2 set into 1 
               t_s = word['start']
               t_e = segment['words'][(j+1)]['end']
               dict_single_log['key_t_s'] = t_s
               dict_single_log['key_t_e'] = t_e
               print("keyword start time: {}, end time: {}".format(t_s, t_e))
               break
         if t_e != 0:
           break
   else:
       # Find the match keyword and its start&end time
       t_s = 0
       t_e = 0
       for i, segment in enumerate(result['segments']):
         #print(segment['words'])
         for j, word in enumerate(segment['words']):
             if search_pattern in word['text'].lower():
               t_s = word['start']
               t_e = word['end']
               dict_single_log['key_t_s'] = t_s
               dict_single_log['key_t_e'] = t_e
               print("keyword start time: {}, end time: {}".format(t_s, t_e))
               break
         if t_e != 0:
           break
   
   # The PCM array's idx           
   rate, data = read('resampled_cut.wav')
   t_s = (int)(np.floor((t_s) * rate)) 
   t_e = (int)(np.ceil((t_e) * rate))
   
   # mean & std, so far no use.
   real_data = np.absolute(data[t_s : t_e])
   mean_d_win = real_data.mean()
   std_win    = real_data.std()
   #print("keyword's mean & std: {}, {}".format(mean_d_win, std_win))
   
   # Add the reduant before & after to a 1 second wav
   tt_buf = (int)((rate*1 - (t_e - t_s))/2)
   t_s -= tt_buf
   t_e += tt_buf
   dict_single_log['key_t_s_adj_idx'] = t_s
   dict_single_log['key_t_e_adj_idx'] = t_e
   #print("array idx of final keyword wav, start, end: {}, {} ".format(t_s, t_e))
   
   # Add zero
   #zero_buf = np.zeros((tt_buf), dtype = int)
   #data = np.concatenate((zero_buf, data[t_s: t_e], zero_buf), axis=None)
   #print(data.size)
   
   # if ma_cut has value, manually cut for minor adjustment
   if ma_cut:
       t_s += (int)(ma_cut['r_shift'] * rate)
       t_e += (int)(ma_cut['r_shift'] * rate)
       dict_single_log['key_t_s_adj_manu'] = t_s
       dict_single_log['key_t_e_adj_manu'] = t_e
       print('manually cut for minor adjustment! {}, {}'.format(t_s, t_e))
   
   # Boundary conditions
   if  t_s < 0:
       t_s = 0
       t_e = (int)(rate)
   elif t_e > len(data) :
       t_e = len(data)
       t_s =  (int)(len(data) - rate)  
   
   output_file = video_id + '_nohash_0.wav'
   write(os.path.join('keyword', output_file), rate, data[t_s: t_e])
   
   return dict_single_log   

def main():
    
    total_line = 0
    with open('yt_dl_list.txt') as f:
        for line in f:
           total_line += 1
           
    with tqdm(total=total_line) as pbar:
        pbar.set_description('Processing segement:')
        
        try:       
            # Start to loop the line in yt list file
            dl_count = 0
            with open('yt_dl_list.txt') as f:
                for line in f:
                    
                   dict_single_log = {} 
                   dl_count += 1 
                   video_url = line.strip()
                   
                   # Extract video ID from the video URL
                   if "youtu.be" in video_url:
                       video_id = video_url.split("youtu.be/")[1]
                   elif "watch?v=" in video_url:    
                       video_id = video_url.split("watch?v=")[1]
                   else:
                       print("The link is error: {}".format(video_url))
                       
                   audio_file = os.path.join('download_raw', (video_id + r'.wav'))     
            
                   if audio_file:
                       # Transcribe audio and grep desired paragraph
                       dict_single_log = transcribe_cut_audio(audio_file, search_pattern, video_id, dict_single_log, {})
                       # print("Done {}!".format(dl_count))
                   else:
                       print("Error downloading audio.")
                   
                   dict_log[video_id] = dict_single_log
                   pbar.update(1)
        except:
            print("The yt_dl_list.txt or func transcribe_cut_audio has issue!")
            
        finally:
            with open("cut_log.json", "w") as outfile:
                json.dump(dict_log, outfile)               

In [26]:
if __name__ == "__main__":
    main()

Processing segement::  20%|██        | 1/5 [00:03<00:15,  3.95s/it]

keyword start time: 6.54, end time: 7.18



Processing segement::  40%|████      | 2/5 [00:06<00:09,  3.30s/it]

keyword start time: 7.26, end time: 7.8



Processing segement::  60%|██████    | 3/5 [00:09<00:06,  3.23s/it]

keyword start time: 4.86, end time: 5.1



Processing segement::  80%|████████  | 4/5 [00:12<00:03,  3.14s/it]

keyword start time: 5.72, end time: 6.06



Processing segement:: 100%|██████████| 5/5 [00:15<00:00,  3.16s/it]

keyword start time: 7.2, end time: 7.96


## 3. Check the format of wav file
- Just make sure the format of keyword wav is correct.

In [27]:

rate, data = read('keyword/4JLNb8-LOB0_nohash_0.wav')

ch = 1 if data.size/data.shape[0]==1 else 2


print("Sample rate: {} Hz".format(rate))
print("Data type: {}".format(data.dtype))
print("Data Seconds: {} s".format(data.shape[0]/rate))
print("Number of channels: {}".format(ch))
print("Data length: {}".format(len(data)))     

Sample rate: 16000 Hz
Data type: int16
Data Seconds: 1.0 s
Number of channels: 1
Data length: 16000


## 4. Minor adjustment the 1(s) window of keyword manually.
- If some keyword wavs have other words overlap, please manually shift the windows.
- Update `video_url` and `Right_Shift_Time` upon your choose. 

In [34]:
video_url = 'https://youtu.be/4JLNb8-LOB0'
Right_Shift_Time = 0.15

In [35]:
ma_cut = {}
ma_cut['r_shift'] = Right_Shift_Time # The unit is (s)


    
# Extract video ID from the video URL
if "youtu.be" in video_url:
    video_id = video_url.split("youtu.be/")[1]
elif "watch?v=" in video_url:    
    video_id = video_url.split("watch?v=")[1]
else:
    print("The link is error: {}".format(video_url))
    
audio_file = os.path.join('download_raw', (video_id + r'.wav'))

if audio_file:
    # Transcribe audio and grep desired paragraph
    dict_single_log = transcribe_cut_audio(audio_file, search_pattern, video_id, {}, ma_cut)
    # print("Done {}!".format(dl_count))
else:
    print("Error downloading audio.")


# Update log
with open('cut_log.json', 'r') as openfile:
    # Reading from json file
    cut_log_json = json.load(openfile)

cut_log_json[video_id]['r_shift'] = Right_Shift_Time  
    
with open("cut_log.json", "w") as outfile:
    json.dump(cut_log_json, outfile)        

100%|██████████| 1400/1400 [00:01<00:00, 1008.89frames/s]

keyword start time: 6.54, end time: 7.18
manually cut for minor adjustment! 104160, 120160
